In [1]:
!pip install spotipy

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached charset_normalizer-3.4.3-cp313-cp313-win_amd64.whl.metadata (37 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.3-cp313-cp313-win_amd64.whl (107 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)

   ---------------------------------------- 0/7 [urllib3]
   ---------------------------------------- 0/7 [urllib3]
   ---------------------------------------- 0/7 [urllib3]
   ---------------------------------------- 0/7 [urllib3]
   ----- ---------------------------------- 1/7 [redis]
   ----- ---------------------------------- 1/7 [redis]
   ----- ---------------------------------- 1/7 [redis]
   ----- ---------------------------------- 1/7 [redis]
   ----- ----------------------------------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install pandas spotipy scikit-learn numpy python-dotenv

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.0 MB 11.7 MB/s eta 0:00:01
   ----------------- ---------------------- 4.7/11.0 MB 11.7 MB/s eta 0:00:01
   -------------------------- ------------- 7.3/11.0 MB 11.7 MB/s eta 0:00:01
   ----------------------------------- ---- 9.7/11.0 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 11.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   --------- ------------------------------ 2.1/8.7 MB 11.7 MB/s eta 0:00:01
   --------------------- ------------------ 4.7/8.7 MB 11.7 MB/s eta 0:00:01
   -------------------------------- ------- 7.1/8.7 MB 11.7 MB/s eta 0:00:01
   ----------------------------------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [1]:
# Make sure to import dotenv and os at the top:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads your .env file variables

# # Then authenticate and print the user
# sp = authenticate_spotify()
# print(sp.current_user())

True

In [2]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

# Load environment variables
load_dotenv()  # ensures .env values are read every run

# Spotify API Authentication
def authenticate_spotify():
    client_id = os.getenv("SPOTIFY_CLIENT_ID")
    client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
    redirect_uri = os.getenv("SPOTIFY_REDIRECT_URI")

    # Delete any cached token to force re-auth with the correct account
    if os.path.exists(".cache"):
        os.remove(".cache")

    # Make sure new cache file is tied to this specific user/session
    return spotipy.Spotify(
        auth_manager=SpotifyOAuth(
            client_id=client_id,
            client_secret=client_secret,
            redirect_uri=redirect_uri,
            scope="user-read-recently-played user-library-read user-top-read",
            cache_path=".cache-" + client_id[-6:],  # unique cache per client
            show_dialog=True  # force Spotify to ask which account to use
        )
    )


# Collect recent tracks
def get_recent_tracks(sp, limit=50):
    results = sp.current_user_recently_played(limit=limit)
    tracks = []
    for item in results['items']:
        track = item['track']
        played_at = item['played_at']  # Timestamp
        name = track['name']
        artist = track['artists'][0]['name']
        track_id = track['id']
        tracks.append([name, artist, track_id, played_at])
    return pd.DataFrame(tracks, columns=['name', 'artist', 'track_id', 'played_at'])


# Get audio features for each track (with safe error handling)
def get_audio_features(sp, track_ids):
    features = []
    for track_id in track_ids:
        try:
            audio_features = sp.audio_features(track_id)[0]
            if audio_features:
                features.append(audio_features)
        except Exception as e:
            print(f"⚠️ Skipping track {track_id} due to error: {e}")
    return pd.DataFrame(features)


# Get artist's genre
def get_artist_genre(sp, artist_name):
    try:
        result = sp.search(q=f'artist:{artist_name}', type='artist')
        if result['artists']['items']:
            return result['artists']['items'][0]['genres']
    except Exception as e:
        print(f"⚠️ Could not fetch genre for {artist_name}: {e}")
    return []


# Extract time of day from the 'played_at' timestamp
def time_of_day(hour):
    if 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    else:
        return 'evening'


# Perform K-means clustering on mood-related features
def perform_clustering(mood_features, n_clusters=3):
    mood_features = mood_features.fillna(0)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    return kmeans.fit_predict(mood_features)


# Get cluster for specific time of day
def get_cluster_for_time_of_day(track_data, time_of_day):
    filtered_data = track_data[track_data['time_of_day'] == time_of_day]
    if not filtered_data.empty:
        return filtered_data['cluster'].mode()[0]
    return None


# Recommend songs from a specific cluster
def recommend_songs_from_cluster(track_data, cluster):
    similar_songs = track_data[track_data['cluster'] == cluster]
    return similar_songs[['name', 'artist']].values.tolist()


# Get recommendations for morning, afternoon, and evening
def recommend_songs_for_times_of_day(track_data):
    time_segments = ['morning', 'afternoon', 'evening']
    recommendations_by_time = {}

    for time in time_segments:
        current_cluster = get_cluster_for_time_of_day(track_data, time)
        if current_cluster is not None:
            recommendations_by_time[time] = recommend_songs_from_cluster(track_data, current_cluster)
        else:
            recommendations_by_time[time] = []
    return recommendations_by_time


# Print the recommendations
def print_recommendations(recommendations):
    for time, songs in recommendations.items():
        print(f"\n🎧 {time.capitalize()} Playlist:")
        for i, song in enumerate(songs, 1):
            print(f"  {i}. {song[0]} by {song[1]}")


# Main function
def main():
    sp = authenticate_spotify()

    # Get recent tracks
    track_data = get_recent_tracks(sp)
    if track_data.empty:
        print("No recent tracks found.")
        return

    # Get audio features
    audio_features = get_audio_features(sp, track_data['track_id'])
    if audio_features.empty:
        print("No audio features found.")
        return

    # Merge track data with audio features
    track_data = pd.concat([track_data, audio_features], axis=1)

    # Add artist genres
    track_data['genre'] = track_data['artist'].apply(lambda x: get_artist_genre(sp, x))

    # Extract time of day
    track_data['played_at'] = pd.to_datetime(track_data['played_at'])
    track_data['hour'] = track_data['played_at'].dt.hour
    track_data['time_of_day'] = track_data['hour'].apply(time_of_day)

    # Select mood-related features
    mood_features = track_data[['valence', 'energy', 'danceability', 'acousticness', 'instrumentalness']]

    # Perform clustering
    track_data['cluster'] = perform_clustering(mood_features)

    # Get recommendations
    recommendations = recommend_songs_for_times_of_day(track_data)

    # Print recommendations
    print_recommendations(recommendations)


# Run main
if __name__ == "__main__":
    main()

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4NeOWqHmlrGRuBvsLJC9rL with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0UHB9METy4VCXNgkcGqHqS with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0hUepWE4GoMwtKgU8vE617 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0hUepWE4GoMwtKgU8vE617 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5AnCLGg35ziFOloEnXK4uu with Params: {} returned 403 due to None


⚠️ Skipping track 4NeOWqHmlrGRuBvsLJC9rL due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=4NeOWqHmlrGRuBvsLJC9rL:
 None, reason: None
⚠️ Skipping track 0UHB9METy4VCXNgkcGqHqS due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=0UHB9METy4VCXNgkcGqHqS:
 None, reason: None
⚠️ Skipping track 0hUepWE4GoMwtKgU8vE617 due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=0hUepWE4GoMwtKgU8vE617:
 None, reason: None
⚠️ Skipping track 0hUepWE4GoMwtKgU8vE617 due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=0hUepWE4GoMwtKgU8vE617:
 None, reason: None
⚠️ Skipping track 5AnCLGg35ziFOloEnXK4uu due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=5AnCLGg35ziFOloEnXK4uu:
 None, reason: None


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6dRVOzSxLqRdqUjc9mctGg with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7y3xU9vEC1s8DSuuoSYKih with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3jVtllWS5CFFWLQng8sKsr with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3Yt9lRtS5V4nbJnwcgFgvC with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5yhKaacMZGRo4s6evJx9vk with Params: {} returned 403 due to None


⚠️ Skipping track 6dRVOzSxLqRdqUjc9mctGg due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=6dRVOzSxLqRdqUjc9mctGg:
 None, reason: None
⚠️ Skipping track 7y3xU9vEC1s8DSuuoSYKih due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=7y3xU9vEC1s8DSuuoSYKih:
 None, reason: None
⚠️ Skipping track 3jVtllWS5CFFWLQng8sKsr due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=3jVtllWS5CFFWLQng8sKsr:
 None, reason: None
⚠️ Skipping track 3Yt9lRtS5V4nbJnwcgFgvC due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=3Yt9lRtS5V4nbJnwcgFgvC:
 None, reason: None
⚠️ Skipping track 5yhKaacMZGRo4s6evJx9vk due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=5yhKaacMZGRo4s6evJx9vk:
 None, reason: None


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5x9uAWpQ0qj8V61xckw6TV with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1EzrEOXmMH3G43AXT1y7pA with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1oTo3ijRbaDAtrjJrGAPSw with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3w3y8KPTfNeOKPiqUTakBh with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2igwFfvr1OAGX9SKDCPBwO with Params: {} returned 403 due to None


...

⚠️ Skipping track 1GhbQDYGEOjyFwfT8lojcx due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=1GhbQDYGEOjyFwfT8lojcx:
 None, reason: None
⚠️ Skipping track 5WQQIDU3HRaMyPkob8mpFb due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=5WQQIDU3HRaMyPkob8mpFb:
 None, reason: None
⚠️ Skipping track 1TEjSXPdAakDotj2Wji3PU due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=1TEjSXPdAakDotj2Wji3PU:
 None, reason: None
⚠️ Skipping track 2vnd3JVvcLs2LxRFwC6mKu due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=2vnd3JVvcLs2LxRFwC6mKu:
 None, reason: None
⚠️ Skipping track 5fGWamKtCGZXW04flUxXl0 due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=5fGWamKtCGZXW04flUxXl0:
 None, reason: None


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3fWibgdtgIie6LKTZ2rUoI with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1EryAkZ0VHstC6haIxVBiE with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=70LcF31zb1H0PyJoS1Sx1r with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2G8PweZBBwTpyP8vpNQJK2 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1YTJW80Jn9Yo1QKKBqEu0e with Params: {} returned 403 due to None


⚠️ Skipping track 3fWibgdtgIie6LKTZ2rUoI due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=3fWibgdtgIie6LKTZ2rUoI:
 None, reason: None
⚠️ Skipping track 1EryAkZ0VHstC6haIxVBiE due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=1EryAkZ0VHstC6haIxVBiE:
 None, reason: None
⚠️ Skipping track 70LcF31zb1H0PyJoS1Sx1r due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=70LcF31zb1H0PyJoS1Sx1r:
 None, reason: None
⚠️ Skipping track 2G8PweZBBwTpyP8vpNQJK2 due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=2G8PweZBBwTpyP8vpNQJK2:
 None, reason: None
⚠️ Skipping track 1YTJW80Jn9Yo1QKKBqEu0e due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=1YTJW80Jn9Yo1QKKBqEu0e:
 None, reason: None


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4KHonMFLzPAc2gqi5JoNu6 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0JbumP59XQzamRG25W67gu with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=493Oixkg3WHKFRdVl8o5OP with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5fGWamKtCGZXW04flUxXl0 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5tlb0AxuzsMWL2GtEppXGX with Params: {} returned 403 due to None


⚠️ Skipping track 4KHonMFLzPAc2gqi5JoNu6 due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=4KHonMFLzPAc2gqi5JoNu6:
 None, reason: None
⚠️ Skipping track 0JbumP59XQzamRG25W67gu due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=0JbumP59XQzamRG25W67gu:
 None, reason: None
⚠️ Skipping track 493Oixkg3WHKFRdVl8o5OP due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=493Oixkg3WHKFRdVl8o5OP:
 None, reason: None
⚠️ Skipping track 5fGWamKtCGZXW04flUxXl0 due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=5fGWamKtCGZXW04flUxXl0:
 None, reason: None
⚠️ Skipping track 5tlb0AxuzsMWL2GtEppXGX due to error: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=5tlb0AxuzsMWL2GtEppXGX:
 None, reason: None
No audio features found.
